# Elope

In [ ]:
def load_images(train, directory):
    
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 224
    train_image = []
    for name in train['image_id']:
        path = directory + 'images/' + name + '.jpg'
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [ ]:
df_test = pd.read_csv(directory + 'test.csv')
x_test = load_images(df_test, directory)

In [ ]:
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.legacy import interfaces

class GlobalKMaxPooling2D(Layer): #Inherits the properties of Layer class

    def __init__(self, data_format=None, k = 10, **kwargs):
        super(GlobalKMaxPooling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        self.k = k

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def get_config(self):
        config = {'data_format': self.data_format, 'k' : self.k}
        base_config = super(GlobalKMaxPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
        
    def call(self, inputs):
        if self.data_format == 'channels_last':
            k = self.k

            input_reshaped = tf.reshape(inputs, [tf.shape(inputs)[0], -1, tf.shape(inputs)[3]])
            input_reshaped = tf.reshape(input_reshaped, [tf.shape(input_reshaped)[0], tf.shape(input_reshaped)[2], tf.shape(input_reshaped)[1]])
            top_k = tf.math.top_k(input_reshaped, k=k, sorted = True, name = None)[0]
            mean = tf.keras.backend.mean(top_k, axis = 2)
            #assert ((input_reshaped.get_shape()[0], input_reshaped.get_shape()[-1]) == mean.get_shape())
        
        return mean

In [ ]:
def load_model(name)
    json_file = open(name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json, custom_objects = {'GlobalKMaxPooling2D': GlobalKMaxPooling2D})
    # load weights into new model
    loaded_model.load_weights(name + ".h5")
    print("Loaded model from disk")
    return loaded_model

In [ ]:
def predict_and_save(model, x_test, name):
    x_pred = model.predict(x_test)
    df_test['healthy'] = x_pred[:,0]
    df_test['multiple_diseases'] = x_pred[:,1]
    df_test['rust'] = x_pred[:,2]
    df_test['scab'] = x_pred[:,3]
    df_test.to_csv(name, index = None)

In [ ]:
model = load_model(name = 'Elope')

In [ ]:
predict_and_save(model, x_test, name = 'Elope_0.csv')

# Batch Confusion Norm